In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Define the architecture of the neural network
input_dim = 10  # example input size
output_dim_sigmoid = 1
output_dim_linear = 3

input_layer = keras.layers.Input(shape=(input_dim, ))
dense_1 = keras.layers.Dense(64, activation='relu')(input_layer)
dense_2 = keras.layers.Dense(32, activation='relu')(dense_1)
output_sigmoid = keras.layers.Dense(output_dim_sigmoid,
                                    activation='sigmoid')(dense_2)
output_linear = keras.layers.Dense(output_dim_linear,
                                   activation='linear')(dense_2)

model = keras.models.Model(inputs=input_layer,
                           outputs=[output_sigmoid, output_linear])

# Compile the model with appropriate optimizer and loss function
model.compile(optimizer='adam',
              loss=['binary_crossentropy', 'mean_squared_error'])

# Generate some dummy data for demonstration purposes
X_train = np.random.rand(100, input_dim)

y_sigmoid = np.random.randint(2, size=(100, output_dim_sigmoid))

y_linear = np.random.randint(0, 256, size=(100, output_dim_linear))

# Train the model on the dummy data
model.fit(X_train, [y_sigmoid, y_linear], epochs=10, batch_size=16)

# Use the model to make predictions on new data
X_new = np.random.rand(1, input_dim)

prediction = model.predict(X_new)

print(prediction)


Epoch 1/10
7/7 [==============================] - 2s 2ms/step - loss: 23798.6699 - dense_30_loss: 0.7160 - dense_31_loss: 23797.9551
Epoch 2/10
7/7 [==============================] - 0s 3ms/step - loss: 23741.5645 - dense_30_loss: 0.7188 - dense_31_loss: 23740.8457
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 23679.7109 - dense_30_loss: 0.7252 - dense_31_loss: 23678.9844
Epoch 4/10
7/7 [==============================] - 0s 3ms/step - loss: 23600.2617 - dense_30_loss: 0.7392 - dense_31_loss: 23599.5234
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 23495.8945 - dense_30_loss: 0.7533 - dense_31_loss: 23495.1426
Epoch 6/10
7/7 [==============================] - 0s 3ms/step - loss: 23363.7441 - dense_30_loss: 0.7763 - dense_31_loss: 23362.9668
Epoch 7/10
7/7 [==============================] - 0s 3ms/step - loss: 23189.5801 - dense_30_loss: 0.7975 - dense_31_loss: 23188.7793
Epoch 8/10
7/7 [==============================] - 0s 10ms/step - loss

In [21]:
sigmoid_output = prediction[0][0]
linear_output = np.array(prediction[1][0])
max_value = np.max(y_linear)
scaled_linear_output = linear_output * (255.0 / max_value)

In [24]:
sigmoid_output = prediction[0][0]
linear_output = np.array(prediction[1][0])
max_value = np.max(y_linear)
scaled_linear_output = linear_output * (255.0 / max_value)

# Print the predicted outputs
print("Sigmoid Output:", sigmoid_output)
print(f'not scale {linear_output}')
print("Linear Output (scaled to 0-255):", scaled_linear_output)


Sigmoid Output: [0.30991226]
not scale [5.627608  4.9225144 5.3218265]
Linear Output (scaled to 0-255): [5.6497636 4.9418945 5.3427787]


In [14]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from miniTransformer.sourcing.sourcing import create_dataset
from miniTransformer.model.GPT_model import GPTModel
from miniTransformer.model.generate import generate_text

# Define the hyperparameters
vocab_size = 34605
embedding_size = 256
num_layers = 4
num_heads = 8
dff = 512
dropout_rate = 0.1
batch_size = 32
num_epochs = 10
max_length= 10

dataset, vocab, char2idx, idx2char, tokenizer = create_dataset('../miniTransformer/data/data.txt', batch_size=32, seq_length=10)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Initialize the GPT model
model = GPTModel(vocab_size, embedding_size, num_layers, num_heads, dff,
                 dropout_rate, max_length)

# Define the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
    # Compute the loss
    loss = loss_object(real, pred)

    # Mask out the padding tokens
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    # Compute the mean loss
    mean_loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)

    return mean_loss


In [16]:
model

In [17]:
import os

# Define the number of iterations between saving the model and generating text
save_freq = 1000
gen_freq = 500

# Define the directory to save the model in
save_dir = './saved_models'

# Create the directory if it does not exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam()

# Define the training metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')

# Iterate over the training data for a fixed number of epochs
for epoch in range(num_epochs):
    # Reset the training metrics
    train_loss.reset_states()

    # Iterate over the batches in the training dataset
    for batch, (x_batch, y_batch) in enumerate(dataset):
        # Open a gradient tape to record the operations
        with tf.GradientTape() as tape:
            # Run the input through the model to get the predictions
            predictions = model(x_batch)

            # Compute the loss between the predictions and the true labels
            loss = loss_function(y_batch, predictions)

        # Compute the gradients of the loss with respect to the model parameters
        gradients = tape.gradient(loss, model.trainable_variables)

        # Apply the gradients to update the model parameters
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Update the training metrics
        train_loss(loss)

        # Print the training progress
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1, batch, train_loss.result()))

        # Save the model every save_freq iterations
        if batch % save_freq == 0:
            model.save(os.path.join(save_dir, 'model_{}_{}.h5'.format(epoch, batch)))

        # Generate a random piece of text every gen_freq iterations
        if batch % gen_freq == 0:
            # Generate a random input sequence
            input_sequence = tf.random.uniform((1, n), dtype=tf.int32, maxval=vocab_size)

            # Generate the output sequence using the model
            output_sequence = generate_text(model, input_sequence, max_length=max_length)

            # Print the output sequence
            print('Generated text:', tokenizer.decode(output_sequence.numpy()[0]))

    # Print the training progress after each epoch
    print('Epoch {} Loss {:.4f}'.format(epoch+1, train_loss.result()))



InvalidArgumentError: Exception encountered when calling layer 'transformer_block_12' (type TransformerBlock).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [32,10,256] vs. [32,10,8] [Op:AddV2]

Call arguments received by layer 'transformer_block_12' (type TransformerBlock):
  • inputs=tf.Tensor(shape=(32, 10, 256), dtype=float32)
  • training=None